<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/finetune_llama2_7b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# https://colab.research.google.com/github/d2l-ai/d2l-tvm-colab/blob/master/chapter_gpu_schedules/arch.ipynb
# T4 GPU
# System RAM 51 GB
# GPU RAM 15 GB
# Disk 166 GB
#
# Success!!!

In [1]:
!pip install transformers torch --quiet
!pip install accelerate peft bitsandbytes transformers trl --quiet

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
import trl

In [ ]:
from trl import SFTTrainer

In [ ]:
#import os
#from torch import cuda, bfloat16
#from time import time
#import transformers
#import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:

#cuda.is_available()

In [4]:
model_name = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-chat-hf", "snapshots", "1")

In [5]:
os.path.exists(model_name)


True

In [6]:
new_model = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-militarywatch")
os.path.exists(new_model)


True

In [11]:


# The instruction dataset to use
dataset_name = "orgcatorg/militarywatchmagazine"



# Tokenizer name
tokenizer_name = "llama-2-7b"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4
per_device_train_batch_size = 1
# Batch size per GPU for evaluation
#per_device_eval_batch_size = 4
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [ ]:
dataset['content'][:2]

In [12]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


In [14]:

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
    print(major)

7


In [15]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb=512'


In [18]:

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


In [ ]:

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


In [ ]:
# https://huggingface.co/datasets/orgcatorg/militarywatchmagazine
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="content",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


In [ ]:
torch.cuda.empty_cache()

In [20]:

# Train model
#trainer.train()
print("done training!")

done training!


In [19]:

# Save trained model
#trainer.model.save_pretrained(new_model)

In [ ]:
print(new_model)

In [21]:
llm = load_dataset(new_model)

In [27]:
llm.data

{'train': MemoryMappedTable
 alpha_pattern: struct<>
 auto_mapping: null
 base_model_name_or_path: string
 bias: string
 fan_in_fan_out: bool
 inference_mode: bool
 init_lora_weights: bool
 layers_pattern: null
 layers_to_transform: null
 lora_alpha: int64
 lora_dropout: double
 modules_to_save: null
 peft_type: string
 r: int64
 rank_pattern: struct<>
 revision: null
 target_modules: list<item: string>
   child 0, item: string
 task_type: string
 ----
 alpha_pattern: [
   -- is_valid: all not null]
 auto_mapping: [1 nulls]
 base_model_name_or_path: [["/content/drive/My Drive/models/Llama-2-7b-chat-hf/snapshots/1"]]
 bias: [["none"]]
 fan_in_fan_out: [[false]]
 inference_mode: [[true]]
 init_lora_weights: [[true]]
 layers_pattern: [1 nulls]
 layers_to_transform: [1 nulls]
 lora_alpha: [[16]]
 ...}

In [ ]:
'''On November 1 deputy chief of the Russian Center for \
 Reconciliation of the Opposing Parties in Syria Rear Admiral Vadim Kulit reported: \
 "The Russian Aerospace Force delivered air strikes on sites of illegal armed formations involved in shelling the positions \
 of the Syrian government forces. A militant strongpoint and an underground shelter were hit in the areas of the settlements \
 of Kansafra and Ein Shib in the Idlib province.” That day also saw two underground bases and a storage facility for drone \
 parts struck, killing an estimated ten militants. The strikes was launched after a Syrian serviceman had been injured during an earlier militant shelling targeting Syrian Army positions in the Abbad district.
'''

In [22]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model

pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=200)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<s>[INST] What the Rear Admiral Vadim Kulit report on November 1? [/INST]  I apologize, but I don't have access to real-time information or current events, including the contents of any reports submitted by Rear Admiral Vadim Kulit. Additionally, I am not able to provide information on individuals or organizations outside of my training data, which primarily focuses on historical and cultural topics.

If you could provide more context or clarify the specific report you are referring to, I may be able to help you better.


In [23]:
prompt = "Provide a detailed report of 200 characters of the conflict between Russia and Ukraine."

In [24]:

result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Provide a detailed report of 200 characters of the conflict between Russia and Ukraine. [/INST]  The conflict between Russia and Ukraine began in 2014 when Russia annexed Crimea, a peninsula in Ukraine. This move was widely condemned by the international community, leading to a series of economic sanctions against Russia. In the following years, pro-Russian separatist movements in eastern Ukraine, backed by Russia, received support from the Russian military, leading to the ongoing Donbas War. The conflict has resulted in thousands of deaths, injuries, and displacement of people, as well as significant economic damage to both countries. Diplomatic efforts to resolve the conflict have been unsuccessful, with tensions remaining high between Russia and Ukraine.
